# Selekcija i evaluacija modela 

Ova sveska sadrži reimplementaciju svih bibliotečkih funkcija koje direktno utiču na selekciju i evaluaciju modela. 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from sklearn import linear_model, svm
from sklearn import model_selection 
from sklearn import metrics
from sklearn import datasets

U radu ćemo, baš kao i u prethodnom primeru, koristiti Viskonsin skup podataka za klasifikaciju tumora na benigne i maligne. 

In [3]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target

Koristićemo i kernelizovani metod potpornih vektora. 

Funkcija `configure_svm_model` je omotač kojim se može konfigurisati kernelizovani model potpornih vektora. Jednostavnosti radi, dozvolićemo samo konfigurisanje vrednost regularizacionog parametra, dok ćemo za vrednost širine kernela uzeti fiksiranu vrednost 0.01.

In [4]:
def configure_svm_model(c):
    return svm.SVC(C=c, gamma=0.01)

Funkcija `classification_error` se može koristiti za ocenu greške klasifikacije.

In [5]:
def classification_error(y_true, y_predicted):
    return 1 - metrics.accuracy_score(y_true, y_predicted)

Funkcija `data_split` deli zadate skupove `X` i `y` na skupove za treniranje i testiranje u razmeri zadatoj parametrom `ratio`. Funkcija vraća skupove `X_train`, `X_test`, `y_train` i `y_test` i oponaša funkcionalnosti bibliotečke `train_test_split` metode.

In [6]:
def data_split(X, y, ratio):
    
    # y.size predstavlja ukupan broj instanci skupa podataka
    # ratio je broj iz intervala [0, 1] i predstavlja pandan train_size parametra bibliotečke funkcije
    # N je ukupan broj instanci skupa za treniranje
    N = int(y.size * ratio)
    
    X_train = X[:N, :]
    y_train = y[:N]
    
    X_test = X[N:, :]
    y_test = y[N:]
    
    return X_train, X_test, y_train, y_test

In [7]:
X_train, X_test, y_train, y_test = data_split(X, y, 0.67)

In [8]:
X_train.shape

(381, 30)

In [9]:
X_test.shape

(188, 30)

Funkcija `train_test_evala` uči zadati model na skupu za treniranje (X_train, y_train), a potom ga evaluira na skupu za testiranje (X_test, y_test) i vraća vrednost funkcije greške `error_function`. Funkcija objedinjuje funkcionalnosti bibliotečkih `fit`, `predict` i `score` metoda.

In [10]:
def train_test_eval(model, X_train, y_train, X_test, y_test, error_function):
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    return error_function(y_test, y_predicted)

Funkcija `train_valid_select` nakon podele skupova `X` i  `y` u razmeri zadatoj parametrom `ratio` na skupove za treniranje i validaciju određuje iz skupa zadatih konfiguracija `configs` onu vrednost konfiguracionog parametra za koju model `configure_model` koji je zadat daje najmanju grešku. Funkcija greške se zadaje parametrom `error_function`.

Funkcija vraća **najbolji** model.

In [11]:
def train_valid_select(X, y, ratio, error_function, configure_model, configs):
    
    X_train, X_validation, y_train, y_validation = data_split(X, y, ratio)
    
    # Za svaku vrednost konfiguracionog parametra izračunavamo grešku modela
    errors = []
    for c in configs:
        model = configure_model(c)
        error = train_test_eval(model, X_train, y_train, X_validation, y_validation, error_function)
        errors.append(error)
        
    errors = np.array(errors)
    
    # Biramo konfiguraciju modela koja ima najmanju grešku
    c_best = configs[np.argmin(errors)]
    
    # Konfigurisemo model za odabrani parametar
    model = configure_model(c_best)
    
    # Treniramo model nad svim podacima
    model.fit(X, y)
    
    # vraćamo najbolji model
    return model

Funkcija `train_valid_test_eval` nakon što podeli podatke `X` i `y` na skupove za treniranje, validaciju i testiranje u razmerama zadatim parametrom `r`, određuje najbolji model korišćenjem `train_valid_select` funkcije i vraća ocenu njegove greške korišćenjem `error_function` funkcije.

In [12]:
def train_valid_test_eval(X, y, ratios, error_function, configure_model, configs):

    # ratios je oblika [r1, r2] 
    # gde r1 predstavlja udeo validacionog skupa, a r2 udeo skupa za testiranje u skupu podataka
    
    # Delimo podatke na podatke za traniranje+validaciju i testiranje
    X_train_validation, X_test, y_train_validation, y_test = data_split(X, y, ratios[0] + ratios[1])

    r = ratios[0] / (ratios[0] + ratios[1])
    
    # Trazimo model sa najboljom konfiguracijom
    model = train_valid_select(X_train_validation, y_train_validation, r, error_function, configure_model, configs)
    
    # Evaluiramo dobijeni model na zasebnom skupu podataka za testiranje
    y_predicted = model.predict(X_test)
    
    # Vracamo model i njegovu gresku
    return model, error_function(y_test, y_predicted)

Skup vrednosti koji se može koristiti za konfiguraciju modela:

In [13]:
configurations = [10**i for i in range(-5, 5)]

Nadalje, tražimo najbolju konfiguraciju za model i vraćamo model koji je obučen sa tom najboljom konfiguracijom. Dalji koraci bi ovde bili da se `model` evaluira na skupu podataka koji do sada **nije viđen niti korišćen**.

In [14]:
model = train_valid_select(X, y, 0.8, classification_error, configure_svm_model, configurations)

In [15]:
print('Najbolji model se dobija za parametar: ', model.C)

Najbolji model se dobija za parametar:  1e-05


Vršimo podelu na skupove za treniranje, validaciju i testiranje. Rezultat rada funkcije je greška modela kao i model obučen na uniji skupova za treniranje i validaciju.

In [16]:
model, error = train_valid_test_eval(X, y, [0.6, 0.2, 0.2], classification_error, configure_svm_model, configurations)
print('Greska: {:.4}'.format(error))
print('Tacnost: {:.4}'.format(1-error))

Greska: 0.2368
Tacnost: 0.7632


## Unakrsna validacija i ugneždjena unakrsna validacija

Funkcija `cross_validation_evaluation` korišćenjem unakrsne validacija evaluira zadati model. Funkcija očekuje model koji se evaluira,  skup podataka, broj slojeva (foldova) na koje se podaci dele i funkciju kojom se računa greška. Funkcija treba da vrati vrednost ukupne greške modela. Funkcija oponaša funkciju `cross_val_score` ali za razliku od nje daje jednu ocenu, a ne više pojedinačnih ocena po slojevima.  

In [17]:
def cross_validation_evaluation(model, X, y, number_of_folds, error_function):
    
    # niz na nivou kojeg ćemo čuvati vrednosti predikcija modela
    y_predicted = np.empty(y.size)
    
    # ix će biti niz skup indeksa koji će se koristiti za generisanje slojeva
    ix = np.arange(0, X.shape[0]) % number_of_folds
    
    # za svaki sloj
    for i in range(number_of_folds):
        # određujemo skup za treniranje
        X_train = X[ix != i, :]
        y_train = y[ix != i]
        
        # oređujemo skup za testiranje
        X_test = X[ix == i, :]
        y_test = y[ix == i]
        
        # treniramo model
        model.fit(X_train, y_train)
        
        # predvidjom vrednosti na skupu za testiranje
        # i cuvamo ih u nizu svih predikcija na odgovarajucim pozicijama
        y_predicted[ix == i] = model.predict(X_test)
        
    # funkcija vraca gresku dobijenu na celom skupu
    return error_function(y, y_predicted)

In [18]:
# Pojašnjenje za liniju: ix = np.arange(0, X.shape[0]) % number_of_folds
# Npr. ako u skupu imamo 20 instanci, a broj slojeva je 4
# naredba ix = np.arange(0, 20) % 4
# ce generisati niz array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])
# u prvom sloju ce se naci sve one instance cije pozicije su obelezene sa 0
# u drugom sloju ce se naci sve one instance cije pozicije su obelezena sa 1 
# i tako redom

### Odabir najbolje konfiguracije korišćenjem unakrsne validacije

Funkcija `cross_validation_selection` korišćenjem unakrsne validacije bira najbolju konfiguraciju za zadati model. Funkcija vraća najbolji model. 

In [19]:
def cross_validation_selection(X, y, number_of_folds, error_function, configure_model, configs):
    
    # niz gresaka modela za razlicite konfiguracije 
    errors = []
    
    # za svaku vrednost parametra u skupu konfiguracija
    for c in configs:
        model = configure_model(c)
        # evaluriamo model unakrsnom validacijom
        error = cross_validation_evaluation(model, X, y, number_of_folds, error_function)
        # i pamtimo gresku u nizu gresaka
        errors.append(error)
        
    # analiziramo za koju vrednost konfiguracionog parametra je dobijena najmanja greska
    errors = np.array(errors)
    c_best = configs[np.argmin(errors)]
    
    # konfigurisemo model 
    model = configure_model(c_best)
    
    # i treniramo ga na celom skupu
    model.fit(X, y)
    
    # ova funkcija vraća najbolji model
    return model

Uočite sličnost sa funkcijom `train_valid_select`.

In [20]:
model = cross_validation_selection(X, y, 10, classification_error, configure_svm_model, configurations)

In [21]:
print('Najbolji model se dobija za parametar: ', model.C)

Najbolji model se dobija za parametar:  10


### Ugnežđena unakrsna validacija

Funkcija `nested_cross_validation_evaluation` korišćenjem ugnježđene unakrsne validacije daje ocenu greške modela.

In [22]:
def nested_cross_validation_evaluation(X, y, number_of_folds, error_function, configure_model, configs):
    
    # niz svih predikcija modela na skupu X
    y_predicted = np.empty(y.size)
    
    # odrejujemo indekse instanci po slojevima
    ix = np.arange(0, X.shape[0]) % number_of_folds
    
    # za svaki sloj
    for i in range(number_of_folds):
        # određujemo trening i validacioni skup
        X_train_validation = X[ix != i]
        y_train_validation = y[ix != i]
        
        # određujemo test skup
        X_test = X[ix == i]
        y_test = y[ix == i]
        
        # određujemo najbolji model unakrsnom validacijom
        model = cross_validation_selection(X_train_validation, y_train_validation,\
                                           number_of_folds, error_function, configure_model, configs)
        
        # čuvamo vrednosti predikcije modela na skupu za testiranje
        y_predicted[ix == i] = model.predict(X_test)
        
    # funkcija vraća grešku na nivou celog skupa
    return error_function(y, y_predicted)

In [23]:
nested_cross_validation_evaluation(X, y, 10, classification_error, configure_svm_model, configurations)

0.37258347978910367